### Kết nối Drive lấy giữ liệu và cài đặt các thư viện cần thiết

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git

In [ ]:
!pip install -r yolov7/requirements.txt

### Nhận diện trực tiếp từ ảnh bằng Yolo v7

In [6]:
%cd /content/yolov7/

/content/yolov7


In [7]:
import os
import sys

import argparse
import time
from pathlib import Path
import cv2
import torch
import numpy as np
import pandas as pd
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel

In [8]:
def letterbox(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True, stride=32):
    shape = img.shape[:2] 
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:
        r = min(r, 1.0)

    ratio = r, r
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1] 
    if auto:
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)
    elif scaleFill:
        dw, dh = 0.0, 0.0
        new_unpad = (new_shape[1], new_shape[0])
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]

    dw /= 2 
    dh /= 2

    if shape[::-1] != new_unpad:
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    return img, ratio, (dw, dh)

In [46]:
opt  = {
    
    "weights": "/content/drive/MyDrive/Fresh_Fruit_Detection/Model/best_final.pt", # Path to weights file default weights are for nano model
    "img-size": 640, # default image size
    "conf-thres": 0.1, # confidence threshold for inference.
    "iou-thres" : 0.45, # NMS IoU threshold for inference.
    "device" : '0',  # device to run our model i.e. 0 or 0,1,2,3 or cpu
    "classes" : None  # list of classes to filter or None

}

In [47]:
def get_image_predict_yolo(img0):
    with torch.no_grad():
        weights, imgsz = opt['weights'], opt['img-size']
        set_logging()
        device = select_device(opt['device'])
        half = device.type != 'cpu'
        model = attempt_load(weights, map_location=device)
        stride = int(model.stride.max()) 
        imgsz = check_img_size(imgsz, s=stride)  
        if half:
            model.half()

        names = model.module.names if hasattr(model, 'module') else model.names
        colors = [[ 255, 127, 44] for _ in names]
        if device.type != 'cpu':
            model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))

        
        img = letterbox(img0, imgsz, stride=stride)[0]
        img = img[:, :, ::-1].transpose(2, 0, 1)  
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  
        img /= 255.0  
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        
        t1 = time_synchronized()
        pred = model(img, augment= False)[0]

        
        classes = None
        if opt['classes']:
            classes = []

            for class_name in opt['classes']:
                classes.append(opt['classes'].index(class_name))


        pred = non_max_suppression(pred, opt['conf-thres'], opt['iou-thres'], classes= classes, agnostic= False)
        t2 = time_synchronized()

        max_cof = 0
        labels_name = ''
        for i, det in enumerate(pred):
            
            gn = torch.tensor(img0.shape)[[1, 0, 1, 0]]
            if len(det):
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()
            
            for *xyxy, conf, cls in reversed(det):
                xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  
                
                label = [int(cls)] + xywh

                labelFrame = f'{names[int(cls)]} - {conf:.2f}'
                labels_name += labelFrame + '\n'
                plot_one_box(xyxy, img0, label=labelFrame, color=colors[int(cls)], line_thickness=3)
    return img0, labels_name

### Build Web

In [35]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [71]:
from PIL import Image
import gradio as gr
import urllib.request

title = """
<div class='header'>
    <div>
        <h4>PBL6 - Khoa học dữ liệu & Trí tuệ nhân tạo</h4>
        <h2 id="title">
            Nhận diện trái cây còn tươi hay hỏng<br>
            Sử dụng mô hình Yolo v7
        </h2>
    </div>
    <img src='https://dut.udn.vn/portals/_default/skins/dhbk/img/front/logo.png'/>
</div>

"""

css = """
img {
    height: 52px;
    width: 320px;
}

.header {
    width: 100%;
    display:flex;
    justify-content: space-between;
}

#title {
        font-size: 30px;
    }
"""

gr_pbl = gr.Blocks(css=css)

def detect(img):
    try:
        img_yolo, labels_name = get_image_predict_yolo(np.asarray(img))
        return Image.fromarray(img_yolo), labels_name
    except:
        urllib.request.urlretrieve('https://media.istockphoto.com/id/1357365823/vector/default-image-icon-vector-missing-picture-page-for-website-design-or-mobile-app-no-photo.jpg?s=612x612&w=0&k=20&c=PM_optEhHBTZkuJQLlCjLz-v3zzxp-1mpNQZsdjrbns=',"no-image.jpeg")
  
        return Image.open("no-image.jpeg"), 'Vui lòng nhập ảnh'

with gr_pbl:
    gr.Markdown(title)

    img_input = gr.Image(type="pil")

    with gr.Row():
        img_output = gr.Image(type="pil")
        label_output = gr.Label(num_top_classes=1)

    detect_btn = gr.Button('Nhận diện')

    detect_btn.click (
        detect,
        inputs = img_input,
        outputs = [img_output, label_output],
        queue=True
    )

gr_pbl.launch(enable_queue=True)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9efafdd0-5e77-4b60.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
